## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-18-55-22 +0000,nypost,Fire chief’s wife allegedly flashed 14-year-ol...,https://nypost.com/2025/11/24/us-news/fire-chi...
1,2025-11-24-18-51-00 +0000,wsj,U.S. to Skip Publication of Initial Third-Quar...,https://www.wsj.com/economy/u-s-to-skip-public...
2,2025-11-24-18-49-55 +0000,nypost,All 31 suspects in bombshell NBA gambling case...,https://nypost.com/2025/11/24/us-news/all-31-s...
3,2025-11-24-18-49-00 +0000,wsj,NASA and Boeing Scale Back Starliner Missions ...,https://www.wsj.com/science/space-astronomy/na...
4,2025-11-24-18-47-07 +0000,nypost,NYC’s Fifth Avenue has never smelled sweeter t...,https://nypost.com/2025/11/24/lifestyle/festiv...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
91,trump,28
182,ukraine,22
185,peace,15
186,plan,14
189,russia,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
178,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,104
177,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,89
37,2025-11-24-17-41-00 +0000,wsj,Ukraine Peace-Plan Negotiators Leave Thorniest...,https://www.wsj.com/world/europe/big-gaps-stil...,87
99,2025-11-24-14-45-38 +0000,nypost,Trump hints ‘something good may just be happen...,https://nypost.com/2025/11/24/us-news/presiden...,87
212,2025-11-23-23-12-21 +0000,nypost,Leaked EU peace deal calls for Ukraine to keep...,https://nypost.com/2025/11/23/world-news/leake...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
178,104,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...
20,50,2025-11-24-18-14-00 +0000,wsj,The Pentagon said it is probing allegations of...,https://www.wsj.com/politics/mark-kelly-senato...
89,45,2025-11-24-15-40-02 +0000,nypost,‘Slender Man’ stabber Morgan Geyser told cops ...,https://nypost.com/2025/11/24/us-news/woman-42...
10,41,2025-11-24-18-33-00 +0000,wsj,President Trump’s advisers are discussing a pr...,https://www.wsj.com/politics/policy/trump-advi...
218,36,2025-11-23-22-31-51 +0000,nypost,Canada’s prime minister says trade talks with ...,https://nypost.com/2025/11/23/business/canadas...
96,34,2025-11-24-14-55-00 +0000,wsj,Kohl’s Names New CEO in Bid to Move Past Leade...,https://www.wsj.com/business/retail/kohls-name...
128,33,2025-11-24-11-58-48 +0000,nypost,Outgoing Rep. Marjorie Taylor Greene shoots do...,https://nypost.com/2025/11/24/us-news/outgoing...
31,30,2025-11-24-17-49-48 +0000,nypost,Judge dismisses cases against ex-FBI Director ...,https://nypost.com/2025/11/24/us-news/judge-di...
28,28,2025-11-24-18-03-20 +0000,nyt,"Jamil Abdullah Al-Amin, Black Power Activist K...",https://www.nytimes.com/2025/11/23/us/h-rap-br...
25,28,2025-11-24-18-08-00 +0000,wsj,The Federal Reserve is struggling to persuade ...,https://www.wsj.com/economy/central-banking/th...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
